In [1]:
cd ..

/home/pierre/git/semisupervised


# Testing probnum

In [2]:
import numpy as np
from probnum.quad import bayesquad
from probnum.quad.integration_measures import IntegrationMeasure, GaussianMeasure

from probnum.quad.solvers import BayesianQuadrature, BQIterInfo
from probnum.quad.typing import DomainLike
from probnum.quad._bayesquad import _check_domain_measure_compatibility

from probnum.randprocs.kernels import Kernel
from probnum.randvars import Normal
from typing import Callable, Optional, Tuple


/home/pierre/git/semisupervised/probnum/randprocs/covfuncs/_covariance_linear_operator.py:16: UserWarning: KeOps is not installed and currently unavailable for Windows.This may prevent scaling to large datasets.
  warnings.warn(


ModuleNotFoundError: No module named 'probnum._version'

In [3]:
def bayesquad_from_initial_data(
    fun: Callable,
    nodes: np.ndarray,
    fun_evals: np.ndarray,
    kernel: Optional[Kernel] = None,
    measure: Optional[IntegrationMeasure] = None,
    domain: Optional[DomainLike] = None,
    policy: Optional[str] = "bmc",
    rng: Optional[np.random.Generator] = None,
    options: Optional[dict] = None
) -> Tuple[Normal, BQIterInfo]:
    r"""Perform Bayesian Quadraturefrom a given set of nodes and function
    evaluations.
    Parameters
    ----------
    nodes
        *shape=(n_eval, input_dim)* -- Locations at which the function evaluations are
        available as ``fun_evals``.
    fun_evals
        *shape=(n_eval,)* -- Function evaluations at ``nodes``.
    kernel
        The kernel used for the GP model. Defaults to the ``ExpQuad`` kernel.
    measure
        The integration measure. Defaults to the Lebesgue measure.
    domain
        The integration domain. Contains lower and upper bound as scalar or
        ``np.ndarray``. Obsolete if ``measure`` is given.
    options
        A dictionary with the following optional solver settings
            scale_estimation : Optional[str]
                Estimation method to use to compute the scale parameter. Defaults
                to 'mle'. Options are
                ==============================  =======
                 Maximum likelihood estimation  ``mle``
                ==============================  =======
            jitter : Optional[FloatLike]
                Non-negative jitter to numerically stabilise kernel matrix
                inversion. Defaults to 1e-8.
    Returns
    -------
    integral :
        The integral belief subject to the provided measure or domain.
    info :
        Information on the performance of the method.
    Raises
    ------
    ValueError
        If neither a domain nor a measure are given.
    Warns
    -----
    UserWarning
        When ``domain`` is given but not used.
    See Also
    --------
    bayesquad : Computes the integral using an acquisition policy.
    Warnings
    --------
    Currently the method does not support tuning of the kernel parameters
    other than the global kernel scale. Hence, the method may perform poorly unless the
    kernel parameters are set to appropriate values by the user.
    """

    if nodes.ndim != 2:
        raise ValueError(
            "The nodes must be given a in an array with shape=(n_eval, input_dim)"
        )

    input_dim, domain, measure = _check_domain_measure_compatibility(
        input_dim=nodes.shape[1], domain=domain, measure=measure
    )

    bq_method = BayesianQuadrature.from_problem(
        input_dim=input_dim,
        kernel=kernel,
        measure=measure,
        domain=domain,
        policy=policy,
        initial_design=None,
        options=options,
    )

    # Integrate
    integral_belief, bqstate, info = bq_method.integrate(
        fun=fun, nodes=nodes, fun_evals=fun_evals, rng=rng
    )

    return integral_belief, bqstate, info

NameError: name 'Callable' is not defined

In [7]:

input_dim = 1
domain = (0, 1)
def fun(x):
    return x.reshape(-1, )
F, info = bayesquad(fun, input_dim, domain=domain, rng=np.random.default_rng(0), policy="ivr_rand")
print(F.mean)


0.500024114110369


In [5]:
nodes = np.linspace(0, 1, 5)[:, None]
nodes = np.array([[10.]])
fun_evals = nodes.reshape(-1, )

fun = lambda x: x.reshape(-1, )
kernel = None
measure = GaussianMeasure(0., 1.)
domain = None  # Obsolete if measure is given
policy = "ivr_rand" #"mi" "ivr" "us" "us_rand", "mi_rand", "ivr_rand"
options = {"max_evals": 10, "batch_size": 5} # 

In [24]:
nodes = np.linspace(0, 1, 5)[:, None]
nodes
nodes = np.array([[10.]])
nodes

array([[10.]])

In [6]:
F, bqstate, info = bayesquad_from_initial_data(fun=fun,nodes=nodes, fun_evals=fun_evals, measure=measure, policy=policy, options=options, rng=np.random.default_rng(0))

In [14]:
info

BQIterInfo(iteration=9, nevals=10, has_converged=True)

In [9]:
bqstate.fun_evals

array([ 1.00000000e+01, -4.45413312e-03,  1.64325161e+00, -1.43428146e+00,
        1.22975381e+00, -8.96526215e-01,  1.92666271e+00, -2.42183301e+00,
        8.49783614e-01, -1.57265056e+00])

In [13]:
F.var

array(4.84997767e-05)

In [ ]:
if nodes.ndim != 2:
    raise ValueError(
        "The nodes must be given a in an array with shape=(n_eval, input_dim)"
    )

input_dim, domain, measure = _check_domain_measure_compatibility(
    input_dim=nodes.shape[1], domain=domain, measure=measure
)

bq_method = BayesianQuadrature.from_problem(
    input_dim=input_dim,
    kernel=kernel,
    measure=measure,
    domain=domain,
    policy=None,
    initial_design=None,
    options=options,
)

# Integrate
integral_belief, bq_state, info = bq_method.integrate(
    fun=None, nodes=nodes, fun_evals=fun_evals, rng=None
)

In [4]:

from torch.distributions.multivariate_normal import MultivariateNormal
import torch


/home/pierre/miniconda3/envs/semisupervised/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Testing the gradients of the quad

In [3]:
cd ..

/home/pierre/git/semisupervised


In [11]:
from module.quadrature import Quadrature
import numpy as np
from probnum.quad import bayesquad
from probnum.quad.integration_measures import IntegrationMeasure, GaussianMeasure

from probnum.quad.solvers import BayesianQuadrature, BQIterInfo
from probnum.quad.typing import DomainLike
from probnum.quad._bayesquad import _check_domain_measure_compatibility
import torch
from probnum.randprocs.kernels import Kernel
from probnum.randvars import Normal
from typing import Callable, Optional, Tuple
from torch.distributions.multivariate_normal import MultivariateNormal


In [5]:
train_x = torch.linspace(0, 1, 5).reshape(-1,1)
train_y = (train_x**2).flatten()
nodes = np.array([[10.]])
fun_evals = nodes.reshape(-1, )

fun = lambda x: x**2
batch_size = 5



dtype = torch.float64
device = "cpu"
kernel = None
measure = GaussianMeasure(0., 1.)
domain = None  # Obsolete if measure is given
policy = "ivr_rand" #"mi" "ivr" "us" "us_rand", "mi_rand", "ivr_rand"
options = {"max_evals": 10, "batch_size": batch_size} # 

mean, loc = torch.zeros(1, dtype=dtype, device=device, requires_grad=True), torch.eye(1, dtype=dtype, device=device, requires_grad=True)
params = [mean, loc]
distribution = MultivariateNormal(loc=params[0], covariance_matrix=params[1])

In [6]:
qd = Quadrature(objective=fun,
            distribution=distribution,
            batch_size=batch_size,
            device=device,
            dtype=dtype,
            policy=policy,
            options=options,
            params=params)

In [7]:
qd.update(train_x, train_y, distribution=distribution)

In [8]:
qd.integrate()

(<Normal with shape=(), dtype=float64>,
 tensor([[-1.4015],
         [-2.3982],
         [ 2.5524],
         [-1.4791],
         [ 1.1725]], dtype=torch.float64),
 tensor([1.9643, 5.7515, 6.5149, 2.1878, 1.3746], dtype=torch.float64))

In [9]:
qd.grad_integration()

ValueError: fun_evals must be one-dimensional (2).

In [15]:
for param in qd.params:
    print(param.grad)

None
None


In [16]:
param.grad.zero_()

AttributeError: 'NoneType' object has no attribute 'zero_'

In [28]:
import torch

x = torch.randn((3,2), requires_grad=True)
out = (x**2).sum(axis=1)
# Pass tensor of ones, each for each item in x
out.backward(torch.ones_like(out))
print(x.grad)
print(f"Reald grad: {2*x}")

tensor([[3.0519, 0.7196],
        [0.9820, 5.0898],
        [2.1712, 0.8690]])
Reald grad: tensor([[3.0519, 0.7196],
        [0.9820, 5.0898],
        [2.1712, 0.8690]], grad_fn=<MulBackward0>)


In [24]:
x = torch.randn((3,2), requires_grad=True)


tensor([[-0.7634, -0.2845],
        [-0.3361, -0.6021],
        [ 1.3452, -0.2483]], requires_grad=True)

tensor([0.6638, 0.4754, 1.8713], grad_fn=<SumBackward1>)

In [ ]:
mean, loc = torch.zeros(1, dtype=dtype, device=device, requires_grad=True), torch.eye(1, dtype=dtype, device=device, requires_grad=True)


In [19]:
mean, loc = torch.zeros(3, dtype=dtype, device=device, requires_grad=True), torch.eye(3, dtype=dtype, device=device, requires_grad=True)
params = [mean, loc]
distribution = MultivariateNormal(loc=params[0], covariance_matrix=params[1])

In [17]:
mean = 100*torch.ones(1, dtype=dtype, device=device, requires_grad=True)

In [20]:
distribution.sample()

tensor([ 0.9337,  0.7861, -0.3251], dtype=torch.float64)

In [22]:
import torch

x = torch.tensor([[1., 2., 3.], [4., 5., 6.]], requires_grad=True)
y = x * x

# Compute gradients for each element of x
z = y.sum()
z.backward()

print(x.grad)

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.]])
